In [8]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
import json
from lark import Lark
import re

In [25]:
def Crawl_One(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.find('div', { 'class' : 'js-store' })['data-content']
    data_json = json.loads(data)
    data_new = {
        'title': data_json['store']['page']['data']['tab']['song_name'],
        'artist': data_json['store']['page']['data']['tab']['artist_name'],
        'key': data_json['store']['page']['data']['tab']['tonality_name'],
        'chords': data_json['store']['page']['data']['tab_view']['wiki_tab']['content']
    }
    return data_new

In [23]:
def Parse_Links(start, stop):
    for i in range(start, stop):
        url = "https://www.ultimate-guitar.com/explore?page=" + str(i) + "&type[]=Chords"
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        data = soup.find('div', { 'class' : 'js-store' })['data-content']
        data_json = json.loads(data)
        links = []
        for tab in data_json['store']['page']['data']['data']['tabs']:
            links.append(tab['tab_url'])
        print(f"Parsed links for page {i}. Current number of links: {len(links)}")
    return links

In [32]:
def Parse_Song(song):
    lines = []
    tag_verification_needed = False
    current_opening_tag = ''
    current_closing_tag = ''
    parsing_opening_tag = False
    parsing_closing_tag = False
    current_content = ''
    parsing_content = False
    current_line = []
    for char in song:
        if char == "[":
            tag_verification_needed = True
            parsing_content = False
            # print('opened a tag - verification needed')
        elif char == "]": # 태그가 닫혔다
            if parsing_opening_tag: # 오프닝태그가 닫혔다
                parsing_opening_tag = False
                if current_opening_tag == 'ch':
                    parsing_content = True
                # print('closed opening tag: ' + current_opening_tag)
            else: # 클로징태그가 닫혔다
                parsing_closing_tag = False
                # print('closed closing tag: ' + current_closing_tag)
                if current_closing_tag == 'ch': # 코드가 닫혔을경우
                    current_line.append(current_content)
                    # print('pushing new chord: ' + current_content)
                    current_opening_tag = ''
                    current_closing_tag = ''
                    current_content = ''
                elif current_closing_tag == 'tab': # 탭이 닫혔을경우
                    lines.append(current_line)
                    # print('pushing new line: ', current_line)
                    current_opening_tag = ''
                    current_closing_tag = ''
                    current_content = ''
                    current_line = []
        elif char == "/":
            if tag_verification_needed:
                tag_verification_needed = False
                parsing_opening_tag = False
                parsing_closing_tag = True
                # print('--tag is closing tag')
            elif parsing_content:
                current_content += char
        else: # 그냥 글자가 왔다
            if char != ' ':
                if tag_verification_needed:
                    if len(current_opening_tag) > 0: # 이미 오프닝태그가 있는 경우 내용 삭제후 새로 파싱
                        current_opening_tag = ''
                        current_content = ''
                        tag_verification_needed = False
                        parsing_opening_tag = True
                        current_opening_tag += char
                    else:
                        tag_verification_needed = False
                        parsing_opening_tag = True
                        parsing_closing_tag = False
                        current_opening_tag += char
                    # print('--tag is opening tag')
                elif parsing_opening_tag:
                    current_opening_tag += char
                elif parsing_closing_tag:
                    current_closing_tag += char
                elif parsing_content:
                    current_content += char
    return lines

In [33]:
def Crawl(start, stop):
    songs = []
    for url in Parse_Links(start, stop):
        print(f'Crawling from url: {url}')
        data = Crawl_One(url)
        data['chords'] = Parse_Song(data['chords'])
        print('Succesfully parsed song.')
        songs.append(data)
    print(songs)
    
Crawl(1,2)

Parsed links for page 1. Current number of links: 50
Crawling from url: https://tabs.ultimate-guitar.com/tab/elvis-presley/cant-help-falling-in-love-chords-1086983
Succesfully parsed song.
Crawling from url: https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589
Succesfully parsed song.
Crawling from url: https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052
Succesfully parsed song.
Crawling from url: https://tabs.ultimate-guitar.com/tab/tom-odell/another-love-chords-1198980
Succesfully parsed song.
Crawling from url: https://tabs.ultimate-guitar.com/tab/eagles/hotel-california-chords-46190
Succesfully parsed song.
Crawling from url: https://tabs.ultimate-guitar.com/tab/radiohead/creep-chords-4169
Succesfully parsed song.
Crawling from url: https://tabs.ultimate-guitar.com/tab/coldplay/yellow-chords-114080
Succesfully parsed song.
Crawling from url: https://tabs.ultimate-guitar.com/tab/vance-joy/riptide-chords-1237247
Succesfully parsed song.
Crawli